## Lab 2a (CS824)

##### This lab and the next (Lab 2b) are based on some excellent material that was developed in 2017 by Barba and Clementi. There seemed little point in reproducing something that they had already clearly laid out with explanations. I have added a few points where I wanted to emphasise certain things or get you to apply some Python skills.

### There are a few sections marked "Exercise" where you need to stop, think and do a little coding. But only the material from Lab 2c needs to be submitted.


###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2017 L.A. Barba, N.C. Clementi

Based on [Module 2 - Lesson 1 from the Barba/Clementi material on Engineering Computation](https://github.com/engineersCode/EngComp2_takeoff) to which I {CWR} have made a few edits.


# Cheers!  Stats with Beers



This module explores practical statistical analysis with Python; this first lesson explores how we can answer questions using data combined with practical methods from statistics.

We'll need some fun data to work with. We found a neat data set of canned craft beers in the US, scraped from the web and cleaned up by Jean-Nicholas Hould ([@NicholasHould](https://twitter.com/NicholasHould?lang=en) on Twitter)—who we want to thank for having a permissive license on his GitHub repository so we can reuse his [work](https://github.com/nickhould/craft-beers-dataset)!

The data source ([@craftcans](https://twitter.com/craftcans) on Twitter) doesn't say that the set includes *all* the canned beers brewed in the country. So we have to asume that the data is a sample and may contain biases.

We'll manipulate the data using **NumPy**—the array library for Python that we learned about in [Module 1, lesson 4](http://go.gwu.edu/engcomp1lesson4). But we'll also learn about a new Python library for data analysis called **pandas**. 

[`pandas`](http://pandas.pydata.org/) is an open-source library providing high-performance, easy-to-use data structures and data-analysis tools.  Even though `pandas` is great for data analysis, we won't exploit all its power in this lesson. But we'll learn more about it later on!

We'll use `pandas` to read the data file (in `csv` format, for comma-separated values), display it in a nice table, and extract the columns that we need—which we'll convert to `numpy` arrays to work with.

Let's start by importing the two Python libraries that we need.

In [4]:
import pandas as pd
import numpy as np

## Step 1: Read the data file

Below, we'll take a peek into the data file, `beers.csv,` using the system command `head` (which we can use with a bang, thanks to IPython).

But first, we will download the data using a Python library for opening a URL on the Internet. We created a short URL for the data file in the public repository with our course materials.

The cell below should download the data in your current working directory. The next cell shows you the first few lines of the data.

In [2]:
from urllib.request import urlretrieve
URL = 'http://go.gwu.edu/engcomp2data1'
urlretrieve(URL, 'beers.csv')

('beers.csv', <http.client.HTTPMessage at 0x7f7a978e8070>)

In [3]:
!head "beers.csv"

,abv,ibu,id,name,style,brewery_id,ounces
0,0.05,,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.09,,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,,2262,Sex and Candy,American IPA,177,12.0
5,0.077,,2261,Black Exodus,Oatmeal Stout,177,12.0
6,0.045,,2260,Lake Street Express,American Pale Ale (APA),177,12.0
7,0.065,,2259,Foreman,American Porter,177,12.0
8,0.055,,2258,Jade,American Pale Ale (APA),177,12.0


{CWR} This illustrates the issue of using command line statements within Jupyter...  i.e. This will work fine if you are in a Linux environment but will throw an error in Windows as there is no built-in 'head' operator in Windows. (Of course once we load the data into a Pandas data-frame then we can use the 'Head' method from Pandas.)

We can use `pandas` to read the data from the `csv` file, and save it into a new variable called `beers`. Let's then check the type of this new variable — we can use the function `type()` to do this.

In [5]:
beers = pd.read_csv("beers.csv")

In [6]:
%whos

Variable      Type         Data/Info
------------------------------------
URL           str          http://go.gwu.edu/engcomp2data1
beers         DataFrame          Unnamed: 0    abv  <...>\n[2410 rows x 8 columns]
np            module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
numpy         module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
pandas        module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
pd            module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
urlretrieve   function     <function urlretrieve at 0x7f7ad440aee0>


In [7]:
type(beers)

pandas.core.frame.DataFrame

This is a new data type for us: a `pandas DataFrame`. From the `pandas` documentation: "A `DataFrame`  is a 2-dimensional labeled data structure with columns of potentially different types" [4]. You can think of it as the contents of a spreadsheet, saved into one handy Python variable. If you print it out, you get a nicely laid-out table: 

In [8]:
beers

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0
...,...,...,...,...,...,...,...,...
2405,2405,0.067,45.0,928,Belgorado,Belgian IPA,424,12.0
2406,2406,0.052,NaN,807,Rail Yard Ale,American Amber / Red Ale,424,12.0
2407,2407,0.055,NaN,620,B3K Black Lager,Schwarzbier,424,12.0
2408,2408,0.055,40.0,145,Silverback Pale Ale,American Pale Ale (APA),424,12.0


By inspecting the table above we can see some aspects of the data. The first column is a numbering scheme for the beers. The other columns contain the following data:

- `abv`: Alcohol-by-volume of the beer.
- `ibu`: International Bittering Units of the beer.
- `id`: Unique identifier of the beer.
- `name`: Name of the beer.
- `style`: Style of the beer.
- `brewery_id`: Unique identifier of the brewery.
- `ounces`: Ounces of beer in the can.

## Step 2: Explore the data

In the field of statistics, [Exploratory Data Analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis) (EDA) has the goal of summarizing the main features of our data, and seeing what the data can tell us without formal modeling or hypothesis-testing. [2]

Let's start by extracting the columns with the `abv` and `ibu` values, and converting them to NumPy arrays. One of the advantages of data frames in `pandas` is that we can access a column simply using its header, like this:

```python
data_frame['name_of_column']
```

The output of this action is a `pandas Series`. From the documentation: "a `Series` is a 1-dimensional labeled array capable of holding any data type." [4]

Check the type of a column extracted by header:

In [12]:
type(beers['name'])

pandas.core.series.Series

Of course, you can index and slice a data series like you know how to do with strings, lists and arrays. Here, we display the first ten elements of the `abv` series:

In [22]:
beers['abv'][2000:2050]

2000    0.047
2001    0.047
2002    0.047
2003    0.062
2004    0.057
2005    0.052
2006    0.050
2007    0.050
2008    0.035
2009    0.049
2010    0.057
2011    0.054
2012    0.050
2013    0.054
2014    0.047
2015    0.047
2016    0.051
2017    0.050
2018    0.050
2019    0.045
2020    0.045
2021    0.051
2022    0.050
2023    0.072
2024    0.050
2025    0.041
2026    0.041
2027    0.032
2028    0.053
2029    0.053
2030    0.047
2031    0.083
2032    0.052
2033    0.054
2034    0.054
2035    0.058
2036    0.083
2037    0.099
2038    0.090
2039    0.053
2040    0.064
2041    0.063
2042    0.064
2043    0.064
2044      NaN
2045    0.090
2046    0.065
2047    0.075
2048    0.056
2049    0.099
Name: abv, dtype: float64

Inspect the data in the table again: you'll notice that there are `NaN` (not-a-number) elements in both the `abv` and `ibu` columns. Those values mean that there was no data reported for that beer. A typical task when cleaning up data is to deal with these `NaN`s.

Let's extract the two series corresponding to the `abv` and `ibu` columns, clean the data by removing all `NaN` values, and then access the values of each series and assign them to a NumPy array. 

In [14]:
abv_series = beers['abv']

In [15]:
len(abv_series)

2410

In [16]:
%whos

Variable      Type         Data/Info
------------------------------------
URL           str          http://go.gwu.edu/engcomp2data1
abv_series    Series       0       0.050\n1       0.<...>gth: 2410, dtype: float64
beers         DataFrame          Unnamed: 0    abv  <...>\n[2410 rows x 8 columns]
np            module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
numpy         module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
pandas        module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
pd            module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
urlretrieve   function     <function urlretrieve at 0x7f7ad440aee0>


Another advantage of `pandas` is that it has the ability to handle missing data. The  data-frame method `dropna()` returns a new data frame with only the good values of the original: all the null values are thrown out. This is super useful!

In [17]:
abv_clean = abv_series.dropna()

Check out the length of the cleaned-up `abv` data; you'll see that it's shorter than the original. The `NaN` entries have gone!

In [18]:
len(abv_clean)

2348

In [24]:
abv_clean[1950:2000]

2004    0.057
2005    0.052
2006    0.050
2007    0.050
2008    0.035
2009    0.049
2010    0.057
2011    0.054
2012    0.050
2013    0.054
2014    0.047
2015    0.047
2016    0.051
2017    0.050
2018    0.050
2019    0.045
2020    0.045
2021    0.051
2022    0.050
2023    0.072
2024    0.050
2025    0.041
2026    0.041
2027    0.032
2028    0.053
2029    0.053
2030    0.047
2031    0.083
2032    0.052
2033    0.054
2034    0.054
2035    0.058
2036    0.083
2037    0.099
2038    0.090
2039    0.053
2040    0.064
2041    0.063
2042    0.064
2043    0.064
2045    0.090
2046    0.065
2047    0.075
2048    0.056
2049    0.099
2050    0.063
2052    0.054
2053    0.071
2054    0.054
2055    0.099
Name: abv, dtype: float64

Remember that a `pandas` _Series_ consists of a column of values, and their labels. You can extract the values via the [`series.values`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.values.html) attribute, which returns a `numpy.ndarray` (multidimensional array). In the case of the `abv_clean` series, you get a one-dimensional array. We save it into the variable name `abv`. 

In [19]:
beers[:10]

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0
5,5,0.077,NaN,2261,Black Exodus,Oatmeal Stout,177,12.0
6,6,0.045,NaN,2260,Lake Street Express,American Pale Ale (APA),177,12.0
7,7,0.065,NaN,2259,Foreman,American Porter,177,12.0
8,8,0.055,NaN,2258,Jade,American Pale Ale (APA),177,12.0
9,9,0.086,NaN,2131,Cone Crusher,American Double / Imperial IPA,177,12.0


In [20]:
abv_clean[:10]

0    0.050
1    0.066
2    0.071
3    0.090
4    0.075
5    0.077
6    0.045
7    0.065
8    0.055
9    0.086
Name: abv, dtype: float64

In [25]:
abv = abv_clean.values

In [26]:
print(abv)

[0.05  0.066 0.071 ... 0.055 0.055 0.052]


In [27]:
type(abv)

numpy.ndarray

In [28]:
%whos

Variable      Type         Data/Info
------------------------------------
URL           str          http://go.gwu.edu/engcomp2data1
abv           ndarray      2348: 2348 elems, type `float64`, 18784 bytes
abv_clean     Series       0       0.050\n1       0.<...>gth: 2348, dtype: float64
abv_series    Series       0       0.050\n1       0.<...>gth: 2410, dtype: float64
beers         DataFrame          Unnamed: 0    abv  <...>\n[2410 rows x 8 columns]
np            module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
numpy         module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
pandas        module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
pd            module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
urlretrieve   function     <function urlretrieve at 0x7f7ad440aee0>


Now we repeat the whole process for the `ibu` column: extract the column into a series, clean it up removing `NaN`s, extract the series values as an array, check how many values we lost.

In [31]:
ibu_series = beers['ibu']

total_len = len(ibu_series)

In [32]:
# After Cleaning
ibu_clean = ibu_series.dropna()

ibu = ibu_clean.values

after_clean = len(ibu)

In [43]:
# Solution to exercise 2a.1
(total_len - after_clean) / total_len *100

41.70124481327801

In [42]:
total_len_abv = len(beers['abv'])
len_after_cleaning_abv = len(abv)

(total_len_abv - len_after_cleaning_abv) / total_len_abv *100

2.572614107883817

### Exercise 2a.1

Write a Python statement that calculates the percentage of missing values for a certain data series. Use the function to calculate the percentage of missing values for the `abv` and `ibu` data sets. 

For the original series, before cleaning, remember that you can access the values with `series.values` (e.g., `abv_series.values`).

##### Outputs:

You should see that in the case of the variable `ibu` we are missing almost 42% of the values. This is important, because it will affect our analysis. When we do descriptive statistics, we will ignore these missing values, and having 42% missing will very likely cause bias.

## Step 3: Ready, stats, go!


Now that we have NumPy arrays with clean data, let's see how we can manipulate them to get some useful information. 

Focusing on the numerical variables `abv` and `ibu`, we'll walk through some "descriptive statistics," below. In other words, we aim to generate statistics that summarize the data concisely.

### Maximum and minimum 

The maximum and minimum values of a dataset are helpful as they tell us the _range_ of our sample: the range gives some indication of the _variability_ in the data.
We can obtain them for our `abv` and `ibu` arrays with the `min()` and `max()` functions from NumPy.

**abv**

In [35]:
abv_min = numpy.min(abv)
abv_max = numpy.max(abv)

In [36]:
print('The minimum value for abv is: ', abv_min)
print('The maximum value for abv is: ', abv_max)

The minimum value for abv is:  0.001
The maximum value for abv is:  0.128


**ibu**

In [37]:
ibu_min = numpy.min(ibu)
ibu_max = numpy.max(ibu)

In [38]:
print('The minimum value for ibu is: ', ibu_min)
print('The maximum value for ibu is: ', ibu_max)

The minimum value for ibu is:  4.0
The maximum value for ibu is:  138.0


### Mean value

The **mean** value is one of the main measures to describe the central tendency of the data: an indication of where's the "center" of the data. If we have a sample of $N$ values, $x_i$, the mean, $\bar{x}$, is calculated by:

\begin{equation*}
    \bar{x} = \frac{1}{N}\sum_{i} x_i
\end{equation*}

In words, that is the sum of the data values divided by the number of values, $N$. 

You've already learned how to write a function to compute the mean in [Module 1 Lesson 5](http://go.gwu.edu/engcomp1lesson5), but you also learned that NumPy has a built-in `mean()` function. We'll use this to get the mean of the `abv` and `ibu` values.

In [39]:
abv_mean = numpy.mean(abv)
ibu_mean = numpy.mean(ibu)

Next, we'll print these two variables, but we'll use some fancy new way of printing with Python's string formatter, `string.format()`. There's a sweet site dedicated to Python's string formatter, called [PyFormat](https://pyformat.info), where you can learn lots of tricks!

The basic trick is to use curly brackets `{}` as placeholder for a variable value that you want to print in the middle of a string (say, a sentence that explains what you are printing), and to pass the variable name as argument to `.format()`, preceded by the string.

Let's try something out…

In [40]:
print('The mean value for abv is {} and for ibu {}'.format(abv_mean, ibu_mean))

The mean value for abv is 0.059773424190800666 and for ibu 42.71316725978647


Ugh! That doesn't look very good, does it? Here's where Python's string formatting gets fancy. We can print fewer decimal digits, so the sentence is more readable. For example, if we want to have four decimal digits, we specify it this way:

In [41]:
print('The mean value for abv is {:.2f} and for ibu {:.2f}'.format(abv_mean, ibu_mean))

The mean value for abv is 0.06 and for ibu 42.71


Inside the curly brackets—the placeholders for the values we want to print—the `f` is for `float` and the `.4` is for four digits after the decimal dot. The colon here marks the beginning of the format specification (as there are options that can be passed before). There are so many tricks to Python's string formatter that you'll usually look up just what you need.
Another useful resource for string formatting is the [Python String Format Cookbook](https://mkaz.blog/code/python-string-format-cookbook/). Check it out!

In [44]:
abv_median = numpy.median(abv)
ibu_median = numpy.median(ibu)
print('The median value for abv is {:.2f} and for ibu {:.2f}'.format(abv_median, ibu_median))

The median value for abv is 0.06 and for ibu 35.00


In [48]:
from scipy import stats
abv_mode = stats.mode(abv)
ibu_mode = stats.mode(ibu)
print(abv_mode)
print(ibu_mode)
print('The modal value for abv is {} and for ibu {}'.format(abv_mode, ibu_mode))

ModeResult(mode=array([0.05]), count=array([215]))
ModeResult(mode=array([20.]), count=array([82]))
The modal value for abv is ModeResult(mode=array([0.05]), count=array([215])) and for ibu ModeResult(mode=array([20.]), count=array([82]))


In [49]:
# The example above demonstrates where you need to check the format of a values returned from a function - in this
# case 'mode' which returns more than just the modal value...

abv_mode = stats.mode(abv)
ibu_mode = stats.mode(ibu)
print('The modal value for abv is {} and for ibu {}'.format(abv_mode, ibu_mode))

The modal value for abv is ModeResult(mode=array([0.05]), count=array([215])) and for ibu ModeResult(mode=array([20.]), count=array([82]))


### Variance and standard deviation

While the mean indicates where's the center of your data, the **variance** and **standard deviation** describe the *spread* or variability of the data. We already mentioned that the _range_ (difference between largest and smallest data values) is also an indication of variability. But the standard deviation is the most common measure of variability.

We really like the way [Prof. Kristin Sainani](https://profiles.stanford.edu/kristin-sainani), of Stanford University, presents this in her online course on [Statistics in Medicine](https://lagunita.stanford.edu/courses/Medicine/MedStats-SP/SelfPaced/about). In her lecture "Describing Quantitative Data: Whhat is the variability in the data?", available [on YouTube](https://youtu.be/hlFeEQF5tDc), she asks: _What if someone were to ask you to devise a statistic that gives the avarage distance from the mean?_ Think about this a little bit.

The distance from the mean, for any data value, is $x_i - \bar{x}$. So what is the average of the distances from the mean? If we try to simply compute the average of all the values $x_i - \bar{x}$, some of which are negative, you'll just get zero! It doesn't work.

Since the problem is the negative distances from the mean, you might suggest using absolute values. But this is just mathematically inconvenient. Another way to get rid of negative values is to take the squares. And that's how we get to the expression for the _variance_: it is the average of the squares of the deviations from the mean. For a set of $N$ values,

\begin{equation*}
     \text{var} = \frac{1}{N}\sum_{i} (x_i - \bar{x})^2
\end{equation*}


The variance itself is hard to interpret. The problem with it is that the units are strange (they are the square of the original units). The **standard deviation**, the square root of the variance, is more meaningful because it has the same units as the original variable. Often, the symbol $\sigma$ is used for it:

\begin{equation*} 
    \sigma = \sqrt{\text{var}} = \sqrt{\frac{1}{N}\sum_{i} (x_i - \bar{x})^2}
\end{equation*}

### Sample vs. population

The above definitions are used when $N$ (the number of values) represents the entire population. But if we have a _sample_ of that population, the formulas have to be adjusted: instead of dividing by $N$ we divide by $N-1$. This is important, especially when we work with real data since usually we have samples of populations. 

The **standard deviation** of a sample is denoted by $s$, and the formula is:

\begin{equation*}     
     s = \sqrt{\frac{1}{N-1}\sum_{i} (x_i - \bar{x})^2}
\end{equation*}

Why? This gets a little technical, but the reason is that if you have a _sample_ of the population, you don't know the _real_ value of the mean, and $\bar{x}$ is actually an _estimate_ of the mean. That's why you'll often find the symbol $\mu$ used to denote the population mean, and distinguish it with the sample mean, $\bar{x}$. Using $\bar{x}$ to compute the standard deviation introduces a small bias: $\bar{x}$ is computed _from the sample values_, and the data are on average (slightly) closer to $\bar{x}$ than the population is to $\mu$. Dividing by $N-1$ instead of $N$ corrects this bias!

Prof. Sainani explains it by saying that we lost one degree of freedom when we estimated the mean using $\bar{x}$.  For example, say we have 100 people and I give you their mean age, and the actual age for 99 people from the sample: you'll be able to calculate the age of that 100th person. Once we calculated the mean, we only have 99 degrees of freedom left because that 100th person's age is fixed. 

### Let's code!

Now that we have the math sorted out, we can program functions to compute the variance and the standard deviation. In our case, we are working with samples of the population of craft beers, so we need to use the formulas with $N-1$ in the denominator. 

In [ ]:
def sample_var(array):
    """ Calculates the variance of an array that contains values of a sample of a 
    population. 
    
    Arguments
    ---------
    array : array, contains sample of values. 
    
    Returns
    -------
    var   : float, variance of the array .
    """
    
    sum_sqr = 0 
    mean = numpy.mean(array)
    
    for element in array:
        sum_sqr += (element - mean)**2
    
    N = len(array)
    var = sum_sqr / (N - 1)
    
    return var
    

Notice that we used `numpy.mean()` in our function: do you think we can make this function even more Pythonic? 

*Hint:* Yes!, we totally can.

### Exercise  2a.2

Re-write the function `sample_var()` using `numpy.sum()` to replace the `for`-loop. Name the function `var_pythonic`.

We have the sample variance, so we take its square root to get the standard deviation. We can make it a function, even though it's just one line of Python, to make our code more readable:

In [ ]:
def sample_std(array):
    """ Computes the standard deviation of an array that contains values
    of a sample of a population.
    
    Arguments
    ---------
    array : array, contains sample of values. 
    
    Returns
    -------
    std   : float, standard deviation of the array.
    """
    
    std = numpy.sqrt(sample_var(array))
    
    return std

Let's call our brand new functions and assign the output values to new variables:

In [ ]:
abv_std = sample_std(abv)
ibu_std = sample_std(ibu)

If we print these values using the string formatter, only printing 4 decimal digits, we can display our descriptive statistics in a pleasant, human-readable way.

In [ ]:
print('The standard deviation for abv is {:.4f} and for ibu {:.4f}'.format(abv_std, ibu_std))

These numbers tell us that the `abv` values are quite concentrated around the mean value, while the `ibu` values are quite spread out from their mean. How could we check these descriptions of the data? A good way of doing so is using graphics: various types of plots can tell us things about the data. 

We'll learn about _histograms_ in this lesson, and in the following lesson we'll explore _box plots_. 

In [ ]:
import statistics as stats
import math

abv_var = numpy.var(abv)
ibu_var = numpy.var(ibu)

abv_sd1 = math.sqrt(abv_var)
ibu_sd1 = math.sqrt(ibu_var)

abv_sd2 = stats.stdev(abv)
ibu_sd2 = stats.stdev(ibu)

print('The variance for abv is {:.4f} and for ibu {:.4f}'.format(abv_var, ibu_var))
print('The standard deviation for abv is {:.4f} and for ibu {:.4f}'.format(abv_sd2, ibu_sd2))

print('A rough estm of standard deviation for abv is {:.4f} and for ibu {:.4f}'.format(abv_sd1, ibu_sd1))


## Step 4: Distribution plots 

Every time that we work with data, visualizing it is very useful. Visualizations give us a better idea of how our data behaves. One way of visualizing data is with a frequency-distribution plot known as **histogram**: a graphical representation of how the data is distributed. To make a histogram, first we need to "bin" the range of values (divide the range into intervals) and then we count how many data values fall into each interval. The intervals are usually consecutive (not always), of equal size and non-overlapping. 

Thanks to Python and Matplotlib, making histograms is easy. We recommend that you always read the documentation, in this case about [histograms](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.hist.html). We'll show you here an example using the `hist()` function from `pyplot`, but this is just a starting point. 

Let's import the libraries that we need for plotting, as you learned in [Module 1 Lesson 5](http://go.gwu.edu/engcomp1lesson5), then study the plotting commands used below. Try changing some of the plot options and seeing the effect.

In [ ]:
from matplotlib import pyplot
%matplotlib inline

#Import rcParams to set font styles
from matplotlib import rcParams

#Set font style and size 
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [ ]:
#You can set the size of the figure by doing:
pyplot.figure(figsize=(10,5))

#Plotting
pyplot.hist(abv, bins=20, color='#3498db', histtype='bar', edgecolor='white') 
#The \n is to leave a blank line between the title and the plot
pyplot.title('abv \n')
pyplot.xlabel('Alcohol by Volume (abv) ')
pyplot.ylabel('Frequency');

In [ ]:
# If you don't specify this then the next plot will be a smaller (default) size
# which can make comparing figures a bit more tricky
# pyplot.figure(figsize=(10,5))

#Plotting
pyplot.hist(ibu, bins=20, color='#e67e22', histtype='bar', edgecolor='white') 
#The \n is to leave a blanck line between the title and the plot
pyplot.title('ibu \n')
pyplot.xlabel('International Bittering Units (ibu)')
pyplot.ylabel('Frequency');

### Exercise  2a.3

Play around with the plots, change the values of the bins, colors, etc.

### Comparing with a normal distribution

A **normal** (or Gaussian) distribution is a special type of distrubution that behaves as shown in the figure: 68% of the values are within one standard deviation $\sigma$ from the mean; 95% lie within $2\sigma$; and at a distance of $\pm3\sigma$ from the mean, we cover 99.7% of the values. This fact is known as the $3$-$\sigma$ rule, or 68-95-99.7 (empirical) rule.

<img src="../images/std_bell_curve.png" style="width: 800px;"/> 
####  Standard deviation and coverage in a normal distribution. Modified figure based on original from [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Standard_deviation_diagram.svg), the free media repository.


Notice that our histograms don't follow the shape of a normal distribution, known as *Bell Curve*. Our histograms are not centered in the mean value, and they are not symetric with respect to it. They are what we call **skewed** to the right (yes, to the _right_). A right (or positive) skewed distribution  looks like it's been pushed to the left: the right tail is longer and most of the values are concentrated on the left of the figure. Imagine that "right-skewed" means that a force from the right pushes on the curve.

### Discuss in your group

* How do you think that skewness will affect the percentages of coverage by standard deviation compared to the Bell Curve?

* Can we calculate those percentages? 

### Exercise  2a.4

Yes we can, and guess what: we can do it in a few lines of Python. But before doing that, we want you to explain in your own words how the following piece of code works. 

*Hints:* 

1. Check what the logical operation `numpy.logical_and(1<x, x<4)` returns.
2. Check what happens if you sum booleans. For example, `True + True`, `True + False` and so on.


In [ ]:
x = numpy.array([1,2,3,4])
num_ele = numpy.logical_and(1<x, x<4).sum()
print(num_ele)

Now, using the same idea, we will calculate the number of elements in each interval of width $(1\sigma, 2\sigma, 3\sigma)$, and get the corresponding percentage. 

### Exercise  2a.5

Since we want to compute this for both of our variables, `abv` and `ibu`, we'll write a function to do so. Discuss the code below in your group - make sure that you understand how it is operating.

In [ ]:
def std_percentages(x, x_mean, x_std):
    """ Computes the percentage of coverage at 1std, 2std and 3std from the
    mean value of a certain variable x.
    
    Arguments
    ---------
    x      : array, data we want to compute on. 
    x_mean : float, mean value of x array.
    x_std  : float, standard deviation of x array.
    
    Returns
    -------
    
    per_std_1 : float, percentage of values within 1 standard deviation.
    per_std_2 : float, percentage of values within 2 standard deviations.
    per_std_3 : float, percentage of values within 3 standard deviations.    
    """
    
    std_1 = x_std
    std_2 = 2 * x_std
    std_3 = 3 * x_std
    
    elem_std_1 = numpy.logical_and((x_mean - std_1) < x, x < (x_mean + std_1)).sum()
    per_std_1 = elem_std_1 * 100 / len(x) 
    
    elem_std_2 = numpy.logical_and((x_mean - std_2) < x, x < (x_mean + std_2)).sum()
    per_std_2 = elem_std_2 * 100 / len(x) 
    
    elem_std_3 = numpy.logical_and((x_mean - std_3) < x, x < (x_mean + std_3)).sum()
    per_std_3 = elem_std_3 * 100 / len(x) 
    
    return per_std_1, per_std_2, per_std_3
    

Let's compute the percentages next. Notice that the function above returns three values. If we want to assign each value to a different variable, we need to follow a specific syntax. In our example this would be:

**abv**

In [ ]:
abv_std1_per, abv_std2_per, abv_std3_per = std_percentages(abv, abv_mean, abv_std)

Let's pretty-print the values of our variables so we can inspect them:

In [ ]:
print('The percentage of coverage at 1 std of the abv_mean is : {:.2f} %'.format(abv_std1_per))
print('The percentage of coverage at 2 std of the abv_mean is : {:.2f} %'.format(abv_std2_per))
print('The percentage of coverage at 3 std of the abv_mean is : {:.2f} %'.format(abv_std3_per))

**ibu**

In [ ]:
ibu_std1_per, ibu_std2_per, ibu_std3_per = std_percentages(ibu, ibu_mean, ibu_std)

In [ ]:
print('The percentage of coverage at 1 std of the ibu_mean is : {:.2f} %'.format(ibu_std1_per))
print('The percentage of coverage at 2 std of the ibu_mean is : {:.2f} %'.format(ibu_std2_per))
print('The percentage of coverage at 3 std of the ibu_mean is : {:.2f} %'.format(ibu_std3_per))

Notice that in both cases the percentages are not that far from the values for normal distribution (68%, 95%, 99.7%), especially for $2\sigma$ and $3\sigma$. So usually you can use these values as a rule of thumb. 

## What we've learned

* Read data from a `csv` file using `pandas`.
* The concepts of Data Frame and Series in `pandas`.
* Clean null (NaN) values from a Series using `pandas`.
* Convert a `panda`s Series into a `numpy` array.
* Compute maximum and minimum, and range.
* Revise concept of mean value.
* Compute the variance and standard deviation.
* Use the mean and standard deviation to understand how the data is distributed.
* Plot frequency distribution diagrams (histograms).
* Normal distribution and 3-sigma rule.


## References

1. [Craft beer datatset](https://github.com/nickhould/craft-beers-dataset) by Jean-Nicholas Hould.
2. [Exploratory Data Analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis), Wikipedia article.
3. _Think Python: How to Think Like a Computer Scientist_ (2012). Allen Downey. Green Tea Press.  [PDF available](http://greenteapress.com/thinkpython/thinkpython.pdf)
4. [Intro to data Structures](https://pandas.pydata.org/pandas-docs/stable/dsintro.html), `pandas` documentation.
5. _Think Stats: Probability and Statistics for Programmers_ version 1.6.0 (2011). Allen Downey. Green Tea Press.  [PDF available](http://greenteapress.com/thinkstats/thinkstats.pdf)
